# Load packages

In [43]:
#Technical
import os
import glob
import zipfile
import pandas as pd
import re
from datetime import datetime
import time
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from tqdm import tqdm
import pickle


zipname = "md_minute.zip"

os.getcwd()

'/Users/aleksandrzybin/Documents/Study/ICEF_MSc2/Big Data and Machine Learning with Applications to Economics and Finance 2021-22/project'

# Data Extraction

In [44]:
#Get unique currency pairs from filenames
with zipfile.ZipFile(zipname, 'r') as f:
    file_names = f.namelist()
    
pair_names = set([item.split("_")[1] for item in file_names])
pair_names = [x for x in pair_names if "USDT" in x]
len(pair_names)

52

In [45]:
#Read csv files and merge them into a df based on the type of currency. Store dfs in a list.
zip_file = zipfile.ZipFile(zipname)
market_data = []
for name in tqdm(pair_names):
    currency_pair = []
    for info in zip_file.infolist():
        if name in info.filename:
            #print(info.filename) See what csv files are gathered into one table
            df =  pd.read_csv(zip_file.open(info.filename), skiprows = 1)
            currency_pair.append(df)
    
    market_data_one_pair = pd.concat(currency_pair, axis=0, ignore_index=True)
    market_data.append(market_data_one_pair)
market_data_dictionary = dict(zip(pair_names, market_data))




  0%|          | 0/52 [00:00<?, ?it/s]


  2%|▏         | 1/52 [00:00<00:27,  1.88it/s]


  4%|▍         | 2/52 [00:03<01:01,  1.22s/it]


  6%|▌         | 3/52 [00:04<00:54,  1.10s/it]


  8%|▊         | 4/52 [00:06<01:16,  1.60s/it]


 10%|▉         | 5/52 [00:07<01:03,  1.36s/it]


 12%|█▏        | 6/52 [00:09<01:13,  1.60s/it]


 13%|█▎        | 7/52 [00:12<01:19,  1.76s/it]


 15%|█▌        | 8/52 [00:12<01:03,  1.44s/it]


 17%|█▋        | 9/52 [00:13<00:53,  1.25s/it]


 19%|█▉        | 10/52 [00:16<01:10,  1.69s/it]


 21%|██        | 11/52 [00:19<01:24,  2.06s/it]


 23%|██▎       | 12/52 [00:22<01:33,  2.35s/it]


 25%|██▌       | 13/52 [00:24<01:35,  2.45s/it]


 27%|██▋       | 14/52 [00:25<01:11,  1.88s/it]


 29%|██▉       | 15/52 [00:25<00:54,  1.47s/it]


 31%|███       | 16/52 [00:26<00:42,  1.19s/it]


 33%|███▎      | 17/52 [00:27<00:40,  1.15s/it]


 35%|███▍      | 18/52 [00:28<00:35,  1.06s/it]


 37%|███▋      | 19/52 [00:29<00:32,  1.02it/s]


 38%|███▊      

In [46]:
market_data_dictionary['ZECUSDT'].head(5)

,unix,date,symbol,open,high,low,close,Volume ZEC,Volume USDT,tradecount
0,1640995140000,2021-12-31 23:59:00,ZEC/USDT,146.5,146.5,146.4,146.5,61.397,8993.7032,38
1,1640995080000,2021-12-31 23:58:00,ZEC/USDT,146.5,146.6,146.2,146.4,150.012,21962.3171,57
2,1640995020000,2021-12-31 23:57:00,ZEC/USDT,146.5,146.7,146.5,146.6,137.229,20110.0472,40
3,1640994960000,2021-12-31 23:56:00,ZEC/USDT,146.4,146.5,146.3,146.4,14.358,2101.8491,21
4,1640994900000,2021-12-31 23:55:00,ZEC/USDT,146.4,146.5,146.4,146.5,0.444,65.0323,2


# Data Summary

In [47]:
names = []
min_time_values = []
max_time_values = []
observations_num_values = []
for key, value in market_data_dictionary.items():
    #get names
    names.append(key)
    #get min/max time
    min_time = datetime.utcfromtimestamp(min(value['unix'])/1000)
    max_time = datetime.utcfromtimestamp(max(value['unix'])/1000) #.strftime('%Y-%m-%d %H:%M:%S') - convert to string date
    min_time_values.append(min_time)
    max_time_values.append(max_time)
    #get number of observations
    observations_num = " ".join([str(round(len(value.index)/10**6, 3)), "mln"])
    observations_num_values.append(observations_num)
general_description = pd.DataFrame({"Currency Pair": names, 
                                   "Starting Date": min_time_values, "Ending Date": max_time_values, 
                                   "Number of obs.": observations_num_values})
general_description

,Currency Pair,Starting Date,Ending Date,Number of obs.
0,AVAXUSDT,2022-04-17 09:52:00,2022-10-02 10:47:00,0.242 mln
1,ETCUSDT,2020-01-16 08:01:00,2022-10-02 08:46:00,1.42 mln
2,AAVEUSDT,2022-01-01 00:01:00,2022-10-02 09:47:00,0.395 mln
3,DASHUSDT,2020-02-04 08:06:00,2022-10-02 08:47:00,1.393 mln
4,ONTUSDT,2022-01-01 00:01:00,2022-10-02 09:48:00,0.395 mln
5,CELRUSDT,2020-09-11 20:41:00,2022-10-02 08:47:00,1.075 mln
6,TUSDUSDT,2020-09-11 20:41:00,2022-09-26 02:59:00,1.066 mln
7,STXUSDT,2022-01-26 11:00:00,2022-10-02 09:48:00,0.358 mln
8,DOTUSDT,2022-01-01 00:01:00,2022-10-02 09:47:00,0.395 mln
9,QTUMUSDT,2020-02-20 08:06:00,2022-10-02 08:46:00,1.369 mln


# Save a dictionary for further use

In [48]:
#Save a dictionary
with open('minute_data.pickle', 'wb') as handle:
    pickle.dump(market_data_dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

#Load a dictionary
#with open('filename.pickle', 'rb') as handle:
#    b = pickle.load(handle)